# Phase 3 Data Analysis

This file is an extension of the Phase 2 Data Cleaning. In this file, we combine what we have from Phase 2 and our hypothesis in order to draw some prediction/conlusion based on the dataset and our observation.

##############################################################################################################################################################################################################

# business

#### - First lets convert the business jason file into a dataframe so we can start the cleaning process

In [1]:
import pandas as pd
import json
business = [json.loads(line) for line in open('yelp_academic_dataset_business.json', 'r')]     # create a dictionary to read in the json objects of the tip file

FileNotFoundError: [Errno 2] No such file or directory: 'yelp_academic_dataset_business.json'

In [ ]:
business_df = pd.json_normalize (business)

#### - After converting the json file to a dataframe and print the first 5 entries, we can see that there are columns that we wont need for our purpose. We also encountered another problem, the business file has all type of business and we are only interested in restaurants. One column that will help us is the "categories" column, lets print the first 20 items in our data to check the results.

#### - What we can see is that "categories" encloses key words describing each business. Since there is no more information that we can use to find if a business is a restaurant, we will find if "food" or "restaurant" is in the "categories" column and filter the results

In [ ]:
all_restaurants_df = business_df[business_df['categories'].str.contains("food") | business_df['categories'].str.contains("Food") | business_df['categories'].str.contains('Restaurant') | business_df['categories'].str.contains('restaurant')] 

### Now, after the filter we ended with a database of 80,505 restaurants. One other thing that we need to do is to delete the columns that we wont use, like the attributes. What we can do is to redo the dataframe again, but this time without copying all the attribute columns

In [ ]:
all_restaurants_df =  all_restaurants_df[['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'stars','review_count','is_open','categories','hours']]
all_restaurants_df.set_index('name', inplace= True)
all_restaurants_df.reset_index(inplace=True)

#### We also need to filter the dataset by countries, since the file contains business from USA and Canada. 

In [ ]:
all_restaurants_df.state.describe()  #to know how many different states do we have in the data

#### Our data has these estates by country:
#### USA: AZ, NV, OH, NC, PA, WI, IL, SC, NY, TX, CO
#### Canada: ON, QC, AB, BC
#### England: XWY, HPL
#### Now we proceed to create two new datasets separated by country for our purposes.

In [ ]:
#Filter the data by canadian restaurants only
canada_restaurants_df=all_restaurants_df[all_restaurants_df['state'].str.contains("ON") | all_restaurants_df['state'].str.contains("QC") | all_restaurants_df['state'].str.contains("AB") | all_restaurants_df['state'].str.contains('BC')]
canada_restaurants_df.reset_index(inplace=True)

In [ ]:
#Filter the data by canadian restaurants only
usa_restaurants_df=all_restaurants_df[all_restaurants_df['state'].str.contains("AZ") | all_restaurants_df['state'].str.contains("CO") | all_restaurants_df['state'].str.contains("TX") | all_restaurants_df['state'].str.contains("NY") | all_restaurants_df['state'].str.contains("SC") | all_restaurants_df['state'].str.contains("IL") | all_restaurants_df['state'].str.contains("WI") | all_restaurants_df['state'].str.contains("PA") | all_restaurants_df['state'].str.contains("NV") | all_restaurants_df['state'].str.contains("OH") | all_restaurants_df['state'].str.contains('NC')]
usa_restaurants_df.reset_index(inplace=True)

## RATES BY STATE

#### Now, we need to create two tables to show how the ratings are distributed in each state by country, and graph these. 

In [ ]:
ratevsstateCA=pd.pivot_table(canada_restaurants_df,index=["state"],columns="stars",values=["name"], aggfunc="count",fill_value=0)
print(ratevsstateCA)
ratevsstateCA.loc[['AB','ON','QC']].plot.bar(title="Rates by state in Canada", fontsize='20', figsize=(30,10))

In [ ]:
ratevsstateUSA=pd.pivot_table(usa_restaurants_df,index=["state"],columns="stars",values="name", aggfunc="count", fill_value=0 )
print (ratevsstateUSA)
ratevsstateUSA.loc[['AZ','IL', 'NC', 'NV','OH','PA','SC','WI']].plot.bar(title="Rates by state in USA", fontsize='20', figsize=(30,10))

### After checking both graphs, we can come up with some observations, like we can see that the most used rating in USA is 4.0 in every state, while in Canada is 3.5. Do this mean that the people in Canada is a little bit more critique? Or that the restaurants in USA provide better service? We will need to make a deeper analysis in phase 3 of the project to prove or disapprove these points.  

## Hamburguer vs Poutine

#### One other thing that we want to analyze is the relation between USA and restaurants that sell hamburguers and Canada with Putine. Since these are the traditional foods for each country, we want to check if these are some of the most popular restaurants based in the number of restaurants and the ratings. 

In [ ]:
import re
import matplotlib.pyplot as plt
burgersUSA= usa_restaurants_df["categories"].str.count("burger", re.I).sum()
burgersUSA= burgersUSA/len(usa_restaurants_df)  #divide by the total number of restaurants in 
#print (burgersUSA)
poutineCA= canada_restaurants_df["categories"].str.count("poutine", re.I).sum()
poutineCA=poutineCA/len(canada_restaurants_df) #divide by the total number of restaurants in canada
#print (poutineCA)
newgraph = [burgersUSA, poutineCA]
plt.bar(["Hamburger in USA", "Poutine in Canada"], newgraph)
plt.title('Hamburger vs Poutine')
plt.ylabel('Restaurants ratio')
plt.show()

#### What we can see from this graph is that the ratio of restaurants that sells burgers in USA is way bigger that the ratio of restaurants that sell poutine in Canada. Since its not common to find restaurants that sell poutine in USA, we can check only how is te ratio in Canada between Hamburgers and Poutin. 

In [ ]:
burgersCA= canada_restaurants_df["categories"].str.count("burger", re.I).sum()
burgersCA= burgersCA/len(canada_restaurants_df)  #divide by the total number of restaurants in 
newgraph = [burgersCA, poutineCA]
plt.bar(["Hamburger", "Poutine"], newgraph)
plt.title('Hamburger vs Poutine in Canada')
plt.ylabel('Restaurants ratio')
plt.show()

#### What we can see in this last graph is that even in Canada hamburgers are more frequent in restaurants than the traditional food "poutine". The total ratio for hamburgers in Canada is just slightly different from the ratio in USA, therefore apparently hamburgers are found in Canada in a similar proportion than in the USA. 

#########################################################################################################################################################################################################################

## Offfical Phase 3 starts from here